# Prepare stats of each captured region  Kronos


##  Input file


The input file for this notebook is a VCF file with the following characteristics:

 * **```ID```** column is in the format ```KronosXXXX.chrYY.NNNNNN``` where 
     * ```XXX``` is the ID of the mutant line
     * ```YY``` is the chromosome where the mutation is
     * ```NNNNNNN``` is the position of the mutation, based on the RefSeq reference. 
 * A single sample named **```sample```**, with at least the following fields. 
    * ```AD``` Allelic depths for the ref and alt alleles in the order listed.
    * ```DP``` Approximate read depth (reads with MQ=255 or with bad mates are filtered)
    

## Call Het/Hom again

Use this criteria: 

 
**HOMOZYGOUS**
HomMinCov3: homozygous mutation present in all reads from an individual and detected at least three times
```
DP>=3
REF=0
ALT>=3
 ```
and if ```ALT>=3```
Kronos: ```ALT/DP >=85%```, then it is a HOM
Cadenza: ```ALT/DP >=90%``` then it is a HOM
 
**HETEROZYGOUS**
HetMinCov5: mutation detected in at least five reads
```DP>=5
REF>=1
ALT>=5
 ```
AND to reduce the probability of calling sequencing errors as heterozygous mutations in regions of high coverage we also determined the minimum percent of mutant reads required for calling a heterozygous mutation.
For Kronos: ```ALT/DP >=15% …. But <85%```
For Cadenza: ```ALT/DP >=10% …. But <90%```
 
If ```ALT/DP <15%``` in Kronos or <10% in Cadenza then they are called wildtype.


## Output file

The script produces a file with all the snps and one with each region covered with SNPs, to be used to identifiy the residual hetorogeneity in a later step. 

| Column | Description                                                                 |
|--------|----------------------------------------------------------------------------|
| CHROM  | Chromosome where the variation is                                          |
| POS    | Initial position of the variation |
| ID     | ID of the SNP, with same convention as described above (```KronosXXXX.chrYY.NNNNNN```) |
| REF    | Reference allele |
| ALT    | Alternative allele |
| LINE   | Linr where the mutation occurs |
| MUT    | Transition of the mutation, it is the same as ```REF>ALT``` |
| EMS    | Is the mutation EMS induced? True when $ MUT \in \{ "G>A", "C>T","A>G","T>C" \} $ |
| HET    | If the mutation is htereozygos or homozygloys **before** applying the rules before |
| DIST   | Distance from the previous variation. This is the value used to break blocks |
| SNP_COUNT       | Variations located in the current region and line. We expected to have 1 at most per line |
| region_length   | length of the current region |
| mut_no          | Number of mutations in the current regions |
| ems_per         | Percentage of the mutations that are actually EMS. We expect a $100\%$ |
| avg_mut_per_pos | Average mutations per position (across lines in the region). Ideally it shoyld be 1 |
| mut_density     | Mutation density of the line in the current region.   $ mut\_density= \frac{100 * mut\_no}{region\_length} $ |
| region_id       | Unique identifierd of the region  |
|region_start    | Start of the region |
|region_end      | End of the region  |
|DP      | Coverage **D**e**p**th |
|RO      | **R**eference allele **O**bservations |
|AO      | **A**lternative allele **O**bservations |
|SNP_INDEX    | $\frac{AO}{DP}$ |
|EMS_GT  |  **Hom**ozygous or **Het**erozygous, based on the rules described above |
|MUT_QUAL  | Quality of the mutation. **HIGH** if the $ ( DP>3 \land EMS\_GT = \text{"Hom"}) \lor ( DP>5 \land EMS\_GT == \text{"Het"} )$. **LOW** if the $DP$ is not enough  |







In [6]:
require 'bio'
require 'zlib'  
require 'bio-vcf'
require 'csv'

false

In [2]:
def print_current_block(out, block )
  len = block.size
  block.each do |vcf_line |
    vcf_line.block_size = len
    out.puts vcf_line.to_block_arr.join("\t")
  end
end

class BioVcf::VcfRecord
    attr_accessor :distance, :snp_count, :block_size, :het 
    @@ems_mut = ["G>A", "C>T","A>G","T>C"]
    
    def mut
        @mut ||= "#{self.ref}>#{self.alt.join(",")}"
    end
    
    def ems?
        @is_ems ||= @@ems_mut.include?(self.mut) 
    end
    
    def line
        @line ||= self.id.split(".")[0]
    end
  
    def chrom_short
        @chrom_short ||= self.chrom.gsub("chr","") 
    end
    
    def to_vcf
        @fields.join("\t") 
    end
    
    def chrom_short!
        @fields[0]=chrom_short
        @chrom = chrom_short
    end
  
    def to_block_arr
        [self.chrom, self.pos, self.id, self.ref, self.alt, self.line,
            self.mut,self.ems?,self.gt, self.distance, self.snp_count, self.block_size]
    end
  
    def snp?
        @snp ||= @fields[3].size == 1 and @fields[4].size == 1
    end
    
    def gt
        @gt ||= self.sample["sample"].gt == "1/1" ? "Hom" : "Het"
    end 
    
    def ad
       @ad ||=  self.sample["sample"].ad
    end
    
    def ao
        ad[1..-1]
    end
    
    def ro
        ad[0]
    end
    
    def dp
        @dp ||= self.sample["sample"].dp
    end
    
    def snp_index
        dp = self.sample["sample"].dp
        return ao.map{|a| a.to_f / dp }
    end
    
    def mut_qual(min_het, max_het)
        return @mut_qual if @mut_qual
        if ems_gt(min_het, max_het) == "Hom"
            if dp >= 3
                @mut_qual = "HIGH"
            elsif dp >= 2
                @mut_qual = "LOW"
            else
               @mut_qual  = "NIL" 
            end
        else
            if dp >= 5
                @mut_qual = "HIGH"
            elsif dp >= 3
                @mut_qual = "LOW"
            else
               @mut_qual  = "NIL" 
            end
        end
        return @mut_qual
    end
    
    def ems_gt(min_het,max_het)    
        return snp_index.map do |s_i|
            case 
            when s_i <= min_het
                return  "Ref"
            when s_i <= max_het
                return  "Het"
            else
                return "Hom"
            end
        end
    end 
end


class BioVcf::VCFfile
    def initialize(file: "", is_gz: true, lines_to_ignore: [])
        @file = file
        @is_gz = is_gz
        @lines_to_ignore = lines_to_ignore
    end
    
    def parseVCFheader(head_line="")
        m=/##INFO=<ID=(.+),Number=(.+),Type=(.+),Description="(.+)">/.match(head_line)
        {:id=>m[1],:number=>m[2],:type=>m[3],:desc=>m[4]}
    end
    
    def each_filter_lines
        return enum_for(:each_filter_lines) unless block_given?
        it = self.each
        while true
            rec = it.next
            yield rec unless @lines_to_ignore.include? rec.line
        end
    end
    
    def each
        return enum_for(:each) unless block_given? 
        if @is_gz
            infile = open(@file)
            io = Zlib::GzipReader.new(infile) 
        else
            io =  File.open(@file)
        end 
        header = BioVcf::VcfHeader.new 
        io.each_line do |line|  
            line.chomp!
            #puts line
            if line =~ /^##fileformat=/     
                header.add(line)  
                while true
                    headerline = io.readline
                    headerline.chomp!
                    unless headerline =~ /^#/
                        line = headerline
                        break #end of header
                    end
                    header.add(headerline)
                end
            end
            
            fields = BioVcf::VcfLine.parse(line)
            rec    = BioVcf::VcfRecord.new(fields,header)
            yield rec
        end
    end
    
    def to_h
        h = Hash.new
        self.each do |vcf_line|
            h[vcf_line.id] = vcf_line
        end
        h
    end
end

:to_h

In [3]:
RegionLineStats = Struct.new(:mut_no, :ems_no, :avg_mut_per_pos, :mut_density)
class SNPRegion
    attr_accessor :id, :fixed
    def initialize(id, fixed:false)
        @block = []
        @line_stats = Hash.new { |h, k| h[k] = RegionLineStats.new(0,0,0,0) }
        @id = id
        @fixed = fixed
    end
    
    def <<(pos)
        enrich_position(pos)
        @block << pos
    end
    
    def enrich_position(pos)
        len = pos.size
        pos.each do |vcf_line|
            ls = @line_stats[vcf_line.line]
            ls.mut_no += 1
            ls.ems_no += 1 if vcf_line.ems?
            vcf_line.snp_count = len
        end
    end
    
    def range(start:0, len:0)
        @range = [start, start + len]
    end

    def region_range
        return @range if @fixed
        range = [0,0]
        range[0] = @block.first[0].pos if @block.first.size > 0
        range[1] = @block.last[0].pos + 1  if @block.last.size > 0
        range
    end
    
    def len
        r = region_range
        r[1] - r[0]
    end

    def iterate_range_by_position
        @block.each do |position |
            position.each do |vcf_line|
                yield vcf_line
            end
        end
    end

    def enrich_region
        return if @block.size == 0 
        range = region_range
        dist = range[1] - range[0]
        size_sum = 0
        @block.each do |position |
            size_sum += position.size
        end
        
        iterate_range_by_position do |vcf_line|
            vcf_line.block_size = dist
        end
        @line_stats.each_pair do |k,region_line|
            region_line.mut_density = 100 * region_line.mut_no.to_f / len
            region_line.avg_mut_per_pos = size_sum.to_f / @line_stats.size #    \/ number of lines 
        end
    end
    
    def print_current_region(out, min_het, max_het)
        iterate_range_by_position do |vcf_line|
            arr = vcf_line.to_block_arr
            region_line = @line_stats[vcf_line.line]
            arr << region_line.mut_no
            arr << region_line.ems_no
            arr << region_line.avg_mut_per_pos
            arr << region_line.mut_density
            arr << self.id
            arr << self.region_range[0]
            arr << self.region_range[1]
            arr << vcf_line.dp
            arr << vcf_line.ro
            arr << vcf_line.ao.join(",")
            arr << vcf_line.snp_index.join(",")
            arr << vcf_line.ems_gt(min_het, max_het)
            arr << vcf_line.mut_qual(min_het, max_het)
            out.puts arr.join("\t")
        end
    end
    
end



:print_current_region

In [4]:
def find_rh_regions(vcf_file, out_prefix, vcf_extra: {} ,region_gap:0, window_size:10000, min_het: 0.15, max_het: 0.85)
  out_file = "#{out_prefix}.tsv.gz"  
  current_chr = ""
  last_position = 0
  i = 0
  prev = 0
  prev_dist = 0
  decide_by = region_gap > 0 ? :region : :window
  out = Zlib::GzipWriter.open(out_file) 
  
  out.puts ["CHROM", "POS", "ID", "REF",
        "ALT", "LINE","MUT","EMS","HET","DIST","SNP_COUNT", "region_length",
        "mut_no", 
        "ems_per", 
        "avg_mut_per_pos",
        "mut_density", "region_id", "region_start", "region_end", "DP","RO","AO","SNP_INDEX", "EMS_GT","MUT_QUAL"].join("\t")
  current_position = Array.new
  reg_id = 0
    
  current_region = SNPRegion.new(reg_id, fixed: decide_by == :window )
  current_region.range(start:0, len:window_size) if decide_by == :window
  last_window = 0
  vcf_file  = BioVcf::VCFfile.new(file:vcf_file )
  i=0
  seen = []
  vcf_file.each do |rec|
    seen << rec.id    
    dist = rec.pos - prev
    current_window = rec.pos/window_size
    dist = rec.pos if dist < 0
    rec.distance = dist
    if dist != 0    
      prev = rec.pos
      current_region  << current_position
      current_position = Array.new
    end
        
    break_block = dist > region_gap if decide_by == :region
    break_block = last_window != current_window if decide_by == :window
        
    if break_block
      current_region.enrich_region
      current_region.print_current_region(out, min_het, max_het)
      reg_id += 1
      current_region = SNPRegion.new(reg_id, fixed: decide_by == :window )
      current_region.range(start:window_size*current_window, len:window_size) if decide_by == :window
    end
        
    current_position << rec
    last_window = current_window
    
  end
  current_region.enrich_region
  current_region.print_current_region(out, min_het, max_het)
  out.close
end



:find_rh_regions

In [5]:
m_file="in/20190709_kronos_merge_nov2018_filter_incomplete.vcf.gz"
find_rh_regions(m_file, "out/20190709_Kronos_snps_with_density_and_het_gap_60",region_gap:60,min_het: 0.15, max_het: 0.85)

#<File:out/20190709_Kronos_snps_with_density_and_het_gap_60.tsv.gz (closed)>